In [1]:
conda install -c plotly plotly=4.1.0

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
#import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

In [3]:
path = "C:\\Users\\sures\\Downloads\\report2.csv"

In [4]:
df = pd.read_csv(path)

In [5]:
df.head()

,EVENT_ID,YEAR,MONTH_NAME,TOR_F_SCALE,STATE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY
0,408409,2012,October,EF0,MISSOURI,37.2772,-93.4429,37.2833,-93.4374,0,0,0,0,25.00K
1,423947,2012,December,EF0,MISSISSIPPI,30.7625,-89.5228,30.7942,-89.5036,0,0,0,0,10.00K
2,417985,2012,December,EF1,ARKANSAS,35.3990,-94.1190,35.4210,-94.0770,0,0,0,0,400.00K
3,359288,2012,March,EF0,SOUTH CAROLINA,34.0720,-81.0760,34.0709,-81.0317,0,0,0,0,30.00K
4,372096,2012,March,EF0,INDIANA,38.8307,-85.0470,38.8312,-85.0460,0,0,0,0,0.00K


In [6]:
repl_dict = {'[kK]': '*1e3', '[mM]': '*1e6', '[bB]': '*1e9', }
df['DAMAGE_PROPERTY'] = df['DAMAGE_PROPERTY'].fillna(1)
df['DAMAGE_PROPERTY'] = df['DAMAGE_PROPERTY'].replace(repl_dict, regex=True).map(pd.eval)

In [7]:
codes = {
    'ALABAMA': 'AL',
    'ALASKA': 'AK',
    'ARIZONA': 'AZ',
    'ARKANSAS': 'AR',
    'CALIFORNIA': 'CA',
    'COLORADO': 'CO',
    'CONNECTICUT': 'CT',
    'DELAWARE': 'DE',
    'DISTRICT OF COLUMBIA': 'DC',
    'FLORIDA': 'FL',
    'GEORGIA': 'GA',
    'HAWAII': 'HI',
    'IDAHO': 'ID',
    'ILLINOIS': 'IL',
    'INDIANA': 'IN',
    'IOWA': 'IA',
    'KANSAS': 'KS',
    'KENTUCKY': 'KY',
    'LOUISIANA': 'LA',
    'MAINE': 'ME',
    'MARYLAND': 'MD',
    'MASSACHUSETTS': 'MA',
    'MICHIGAN': 'MI',
    'MINNESOTA': 'MN',
    'MISSISSIPPI': 'MS',
    'MISSOURI': 'MO',
    'MONTANA': 'MT',
    'NEBRASKA': 'NE',
    'NEVADA': 'NV',
    'NEW HAMPSHIRE': 'NH',
    'NEW JERSEY': 'NJ',
    'NEW MEXICO': 'NM',
    'NEW YORK': 'NY',
    'NORTH CAROLINA': 'NC',
    'NORTH DAKOTA': 'ND',
    'NORTHERN MARIANA ISLANDS':'MP',
    'OHIO': 'OH',
    'OKLAHOMA': 'OK',
    'OREGON': 'OR',
    'PALAU': 'PW',
    'PENNSYLVANIA': 'PA',
    'PUERTO RICO': 'PR',
    'RHODE ISLAND': 'RI',
    'SOUTH CAROLINA': 'SC',
    'SOUTH DAKOTA': 'SD',
    'TENNESSEE': 'TN',
    'TEXAS': 'TX',
    'UTAH': 'UT',
    'VERMONT': 'VT',
    'VIRGIN ISLANDS': 'VI',
    'VIRGINIA': 'VA',
    'WASHINGTON': 'WA',
    'WEST VIRGINIA': 'WV',
    'WISCONSIN': 'WI',
    'WYOMING': 'WY',
}

In [9]:
# Group by and aggregate over property damage by state
df_property_damage = df.groupby(['STATE'])[['DAMAGE_PROPERTY']].sum()
df_property_damage = df_property_damage.reset_index()

fig = go.Figure(data=go.Choropleth(
    locations=df_property_damage['STATE'].apply(lambda x: codes[x]), # Replace state names with code
    z = df_property_damage['DAMAGE_PROPERTY'].astype(float), 
    locationmode = 'USA-states',
    colorscale = 'Reds',
    colorbar_title = "Billions USD",
    text = df_property_damage['STATE']
))


fig.update_layout(
    title_text = 'Property Damage caused by Tornado Events by State in Years 2012-2016',
    geo_scope='usa', 
)

fig.show()

In [10]:
# Group by and count over event id by state
df_tornado_counts = df.groupby(['STATE'])[['EVENT_ID']].count()
df_tornado_counts = df_tornado_counts.reset_index()

fig = go.Figure(data=go.Choropleth(
    locations=df_tornado_counts['STATE'].apply(lambda x: codes[x]), # Replace state names with code
    z = df_tornado_counts['EVENT_ID'].astype(float), 
    locationmode = 'USA-states',
    colorscale = 'Blues',
    colorbar_title = "Number of Tornado Events",
    text = df_property_damage['STATE']
))

fig.update_layout(
    title_text = 'Count of Tornado Events by State in Years 2012-2016',
    geo_scope='usa', 
)

fig.show()

In [11]:
# Filter dataframe by significant tornado events (EF2,EF3,EF4,EF5)
df_significant_tornado = df[df['TOR_F_SCALE'].isin(['EF2','EF3','EF4','EF5'])]

df_significant_tornado_count = df_significant_tornado.groupby(['STATE'])[['EVENT_ID']].count()
df_significant_tornado_count = df_significant_tornado_count.reset_index()

fig = go.Figure(data=go.Choropleth(
    locations=df_significant_tornado_count['STATE'].apply(lambda x: codes[x]), # Replace state names with code
    z = df_significant_tornado_count['EVENT_ID'].astype(float),
    locationmode = 'USA-states',
    colorscale = 'Blues',
    colorbar_title = "Number of Tornado Events",
))

fig.update_layout(
    title_text = 'Count of significant Tornado Events (EF Scale >= 2) by State in Years 2012-2016',
    geo_scope='usa', # limite map scope to USA
)

fig.show()
